In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import cross_val_score

## Análise Inicial

In [301]:
df = pd.read_csv("dataset/kaggle_train.csv")


In [302]:
df.head()

,Id,description,address,property_type,price,size,bedrooms,bathrooms,parking_spaces,contract_type,latitude,longitude
0,3796,"SGAN 906 Módulo E, ASA NORTE, BRASILIA",NaN,kitnet,270000.0,27.0,1.0,NaN,NaN,venda,-15.771441,-47.894002
1,4830,"QNG 2, TAGUATINGA NORTE, TAGUATINGA",NaN,casa,520000.0,249.0,4.0,NaN,5.0,venda,-15.799918,-48.060482
2,1521,"Rua 4C Chacará 14, VICENTE PIRES, VICENTE PIRES",NaN,apartamento,285000.0,98.0,2.0,NaN,2.0,venda,-15.817467,-48.024206
3,13571,"Residencial Damha I Quadra A1, RESIDENCIAL E C...",NaN,lote,120000.0,380.0,NaN,NaN,NaN,venda,-15.797515,-47.891887
4,4541,"GRANJA DO TORTO, GRANJA DO TORTO, BRASILIA",NaN,casa,780000.0,300.0,4.0,NaN,3.0,venda,-15.703169,-47.912267


In [303]:
df.columns

Index(['Id', 'description', 'address', 'property_type', 'price', 'size',
       'bedrooms', 'bathrooms', 'parking_spaces', 'contract_type', 'latitude',
       'longitude'],
      dtype='object')

In [304]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12194 entries, 0 to 12193
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              12194 non-null  int64  
 1   description     12194 non-null  object 
 2   address         0 non-null      float64
 3   property_type   12194 non-null  object 
 4   price           12194 non-null  float64
 5   size            11590 non-null  float64
 6   bedrooms        9632 non-null   float64
 7   bathrooms       0 non-null      float64
 8   parking_spaces  7933 non-null   float64
 9   contract_type   12194 non-null  object 
 10  latitude        12194 non-null  float64
 11  longitude       12194 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 1.1+ MB


In [305]:
df.describe()

,Id,address,price,size,bedrooms,bathrooms,parking_spaces,latitude,longitude
count,12194.000000,0.0,12194.000000,11590.000000,9632.000000,0.0,7933.000000,12194.000000,12194.000000
mean,7627.375431,NaN,472782.505248,444.818550,3.021595,NaN,2.758225,-15.862162,-47.975199
std,4402.557957,NaN,246909.622836,4623.596605,1.324415,NaN,2.507233,0.461063,0.260009
min,2.000000,NaN,100000.000000,1.000000,1.000000,NaN,1.000000,-23.758431,-49.271470
25%,3824.250000,NaN,267000.000000,78.000000,2.000000,NaN,1.000000,-15.880037,-48.070185
50%,7594.500000,NaN,420000.000000,154.500000,3.000000,NaN,2.000000,-15.834478,-48.012380
75%,11458.750000,NaN,660000.000000,280.000000,3.000000,NaN,3.000000,-15.797515,-47.913309
max,15243.000000,NaN,999999.000000,250000.000000,22.000000,NaN,60.000000,-3.767821,-38.663459


In [306]:
df["property_type"].value_counts()

property_type
casa           6323
apartamento    2914
lote           2111
rural           560
kitnet          143
sala            143
Name: count, dtype: int64

In [307]:
df["contract_type"].value_counts()

contract_type
venda    12194
Name: count, dtype: int64

In [308]:
df.select_dtypes(exclude=['object']).corr()["price"].sort_values(ascending=False)

price             1.000000
bedrooms          0.393282
parking_spaces    0.255736
Id                0.080621
longitude         0.056772
latitude          0.051069
size              0.038135
address                NaN
bathrooms              NaN
Name: price, dtype: float64

### Feature engineering

In [309]:
df = df.drop(["bathrooms", "address", "contract_type"], axis=1)
df

,Id,description,property_type,price,size,bedrooms,parking_spaces,latitude,longitude
0,3796,"SGAN 906 Módulo E, ASA NORTE, BRASILIA",kitnet,270000.0,27.0,1.0,NaN,-15.771441,-47.894002
1,4830,"QNG 2, TAGUATINGA NORTE, TAGUATINGA",casa,520000.0,249.0,4.0,5.0,-15.799918,-48.060482
2,1521,"Rua 4C Chacará 14, VICENTE PIRES, VICENTE PIRES",apartamento,285000.0,98.0,2.0,2.0,-15.817467,-48.024206
3,13571,"Residencial Damha I Quadra A1, RESIDENCIAL E C...",lote,120000.0,380.0,NaN,NaN,-15.797515,-47.891887
4,4541,"GRANJA DO TORTO, GRANJA DO TORTO, BRASILIA",casa,780000.0,300.0,4.0,3.0,-15.703169,-47.912267
...,...,...,...,...,...,...,...,...,...
12189,5192,"Rua 11, SAO FRANCISCO, SAO SEBASTIAO",casa,810000.0,120.0,3.0,1.0,-23.758431,-45.415684
12190,13419,"QS 417 Conjunto A, SAMAMBAIA NORTE, SAMAMBAIA",lote,300000.0,200.0,NaN,NaN,-15.882325,-48.118433
12191,5391,"Quadra 32, CHACARAS QUEDAS DO DESCOBERTO, AGUA...",casa,450000.0,200.0,3.0,3.0,-15.768771,-48.271678
12192,861,"QN 304, SAMAMBAIA SUL, SAMAMBAIA",apartamento,269000.0,48.0,2.0,1.0,-15.876854,-48.080946


In [310]:
desc = df['description'].str.split(',', expand=True)
df['RA'] = desc.apply(lambda row: next((str(row[i]).strip() for i in range(3, -1, -1) if pd.notnull(row[i])), None), axis=1)
df.head()

,Id,description,property_type,price,size,bedrooms,parking_spaces,latitude,longitude,RA
0,3796,"SGAN 906 Módulo E, ASA NORTE, BRASILIA",kitnet,270000.0,27.0,1.0,NaN,-15.771441,-47.894002,BRASILIA
1,4830,"QNG 2, TAGUATINGA NORTE, TAGUATINGA",casa,520000.0,249.0,4.0,5.0,-15.799918,-48.060482,TAGUATINGA
2,1521,"Rua 4C Chacará 14, VICENTE PIRES, VICENTE PIRES",apartamento,285000.0,98.0,2.0,2.0,-15.817467,-48.024206,VICENTE PIRES
3,13571,"Residencial Damha I Quadra A1, RESIDENCIAL E C...",lote,120000.0,380.0,NaN,NaN,-15.797515,-47.891887,CIDADE OCIDENTAL
4,4541,"GRANJA DO TORTO, GRANJA DO TORTO, BRASILIA",casa,780000.0,300.0,4.0,3.0,-15.703169,-47.912267,BRASILIA


In [311]:
df = df.join(pd.get_dummies(df["RA"], dtype=int))
df = df.join(pd.get_dummies(df["property_type"], dtype=int))
df = df.drop(columns=["description", "RA", "property_type"])

In [312]:
df.head()

,Id,price,size,bedrooms,parking_spaces,latitude,longitude,AGUAS CLARAS,AGUAS LINDAS DE GOIAS,ALPHAVILLE,...,VALPARAISO DE GOIAS,VARJAO,VICENTE PIRES,VILA ESTRUTURAL,apartamento,casa,kitnet,lote,rural,sala
0,3796,270000.0,27.0,1.0,NaN,-15.771441,-47.894002,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,4830,520000.0,249.0,4.0,5.0,-15.799918,-48.060482,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1521,285000.0,98.0,2.0,2.0,-15.817467,-48.024206,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,13571,120000.0,380.0,NaN,NaN,-15.797515,-47.891887,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4541,780000.0,300.0,4.0,3.0,-15.703169,-47.912267,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [313]:
def init_x_test(df):
    desc = df['description'].str.split(',', expand=True)
    df = df.drop(["bathrooms", "address", "contract_type"], axis=1)
    df['RA'] = desc.apply(lambda row: next((str(row[i]).strip() for i in range(3, -1, -1) if pd.notnull(row[i])), None), axis=1)
    df = df.join(pd.get_dummies(df["RA"], dtype=int))
    df = df.join(pd.get_dummies(df["property_type"], dtype=int))
    df = df.drop(["description", "property_type", "RA"], axis=1)
    return df

# Treinando o Modelo

In [314]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [315]:
X_train = df.loc[:, df.columns != "price"]
Y_train = df["price"]

X_test = pd.read_csv("dataset/kaggle_test.csv")
X_test = init_x_test(X_test)

In [316]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12194 entries, 0 to 12193
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Id                           12194 non-null  int64  
 1   size                         11590 non-null  float64
 2   bedrooms                     9632 non-null   float64
 3   parking_spaces               7933 non-null   float64
 4   latitude                     12194 non-null  float64
 5   longitude                    12194 non-null  float64
 6   AGUAS CLARAS                 12194 non-null  int64  
 7   AGUAS LINDAS DE GOIAS        12194 non-null  int64  
 8   ALPHAVILLE                   12194 non-null  int64  
 9   BRASILIA                     12194 non-null  int64  
 10  BRAZLANDIA                   12194 non-null  int64  
 11  CANDANGOLANDIA               12194 non-null  int64  
 12  CEILANDIA                    12194 non-null  int64  
 13  CIDADE OCIDENTAL

In [ ]:
X_test.head()

# Random Forest

In [317]:
model = RandomForestRegressor()
model.fit(X_train, Y_train)

predicts = model.predict(X_train)

In [318]:
scores = cross_val_score(model, X_train, Y_train, scoring="neg_mean_squared_error", cv=10)
model_score = np.sqrt(-scores)

print(f"Scores {model_score}")
print(f"Mean {model_score.mean()}")
print(f"Standart deviation {model_score.std()}")

Scores [105169.08960388  99847.92468531 102553.89227687 104779.35770121
 101269.23931038  95519.17544786  92376.54694138 105939.40951269
 103446.5374887  111629.58511082]
Mean 102253.07580791033
Standart deviation 5172.542736137748


# Avaliação e Otimização


In [319]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = [
    {
        'n_estimators': [3, 10, 30, 50, 100], 
        'max_features': [2, 4, 6, 8, 10, 12], 
        'max_depth': [None, 10, 20, 30], 
        'min_samples_split': [2, 5, 10], 
        'min_samples_leaf': [1, 2, 4]
    },
    {
        'n_estimators': [3, 10, 30, 50], 
        'max_features': [2, 3, 4, 6, 8], 
        'bootstrap': [False], 
        'max_depth': [None, 10, 20], 
        'min_samples_split': [2, 5], 
        'min_samples_leaf': [1, 2]
    }
]

model = RandomForestRegressor()


grid_search = GridSearchCV(model, param_grid, cv= 5, scoring="neg_mean_squared_error")

grid_search.fit(X_train, Y_train)

In [ ]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 4, 'n_estimators': 10}

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, max_features=4, n_estimators=10)

In [ ]:
model = grid_search.best_estimator_ 
predictions = model.predict(X_test)
predictions

print(f"Scores {model_score}")
print(f"Mean {model_score.mean()}")
print(f"Standart deviation {model_score.std()}")

Scores [106092.98158444  99981.59711154 104515.81902204 106744.67039335
 101021.69433866  94983.18612585  92533.46568669 105528.80593102
 103444.42489113 110913.34900443]
Mean 102575.9994089145
Standart deviation 5292.868727159963


In [ ]:
predictions = pd.DataFrame({'Id': X_test['Id'], 'price': predictions})

predictions.to_csv('dataset/predictions.csv', index=False)

ValueError: Data must be 1-dimensional, got ndarray of shape (3049, 2) instead